In [1]:
# created on Dec 24, 2020
# modified on April 25, 2022
# @author:          Ajaz Syed
# @email:           aks3872@uw.edu
# @organization:    University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [2]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [11]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access

    # Removed the access token as said in the instructions


    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONSUS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii



    LOCATIONSUK = [-5.907002, 49.560815, -0.553241, 60.794147] # UK
     

    stream_listener = StreamListener(time_limit=180, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONSUK, languages=['en'], encoding="utf-8")

{'id': 1519038692526071808, 'username': 'ralphymoon', 'created_at': 'Tue Apr 26 19:40:21 +0000 2022', 'lng': -3.055158, 'lat': 56.495491, 'text': '@conarkerr1996 @DundeeStars Sorted now 👌🏻'}
{'id': 1519038693163651074, 'username': 'VonnSand', 'created_at': 'Tue Apr 26 19:40:21 +0000 2022', 'lng': -4.2417514999999995, 'lat': 55.85830249999999, 'text': '@justjanis Janis I have ZERO memory of this so I am willing to bet it was absolute mince from start to finish haha'}
{'id': 1519038698184192000, 'username': 'greg_curious', 'created_at': 'Tue Apr 26 19:40:22 +0000 2022', 'lng': -3.2030085, 'lat': 55.9431955, 'text': '@scottishgreens @patrickharvie @lornaslater Sell outs on the big bucks.Pie Paddy &amp; Lazy/Loopy Lorna.  From a part… https://t.co/2n7S1SlWdn'}
{'id': 1519038700201594880, 'username': 'MrsJoJoM13', 'created_at': 'Tue Apr 26 19:40:23 +0000 2022', 'lng': -2.0093330000000003, 'lat': 52.514873, 'text': '@misssimmo78 @MrWithersAHT Me too, it was so good. The lecturers were great,

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
